In [ ]:
import src.alex.reaction_diffusion as rd
import src.noa.monte_carlo_DLA as dla
import numpy as np

# Part I

In [ ]:
dla.plot_dla()

In [ ]:
# sticking probabilities
ps = np.round(np.arange(0.005, 1, 0.05), 3)
N = 15

# plot metrics to compare resulting clusters the sticking probabilities
# dla.generate_cluster_data(ps,N)
rgs, rgs_std, fractal_dims, fractal_dims_std = dla.data_analysis(ps,N)
dla.plot_data_analysis(ps, rgs, rgs_std, fractal_dims, fractal_dims_std)

# Part II

In [ ]:
# your presentation code here

# Part III

In [ ]:
# plot_html is for ipynb animation showing
rd.reset_global_UV()
rd.plot_html(boundary_condition="PBC", observed_variable = "V", f_value = 0.022, k_value = 0.051)

In [ ]:
# plot_html is for ipynb animation showing
rd.reset_global_UV()
rd.plot_html(boundary_condition="PBC", observed_variable = "V", f_value = 0.030, k_value = 0.060)

In [ ]:
# plot_html is for ipynb animation showing
rd.reset_global_UV()
rd.plot_html(boundary_condition="PBC", observed_variable = "V", f_value = 0.047, k_value = 0.063)

In [ ]:
# plot_animation is for saving the animation as a mp4 in the fig folder

# f = 0.022, k = 0.051
# f = 0.030, k = 0.060
# f = 0.047, k = 0.063
fkSet = [
    {0.022, 0.051},
    {0.030, 0.060},
    {0.047, 0.063}
    ]

for v in fkSet:
    f = v.pop()
    k = v.pop()
    
    # periodic boundary condition
    rd.plot_animation("PBC", observed_variable = "V", f_value = f, k_value = k)

    # Dirichlet boundary condition
    rd.plot_animation("Dirichlet Strong", observed_variable = "V", f_value = f, k_value = k)